In [1]:
import pandas as pd
import telebot
import threading
import time
import logging
import os

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Replace 'YOUR_BOT_TOKEN' with your actual bot token
bot_token = "7498318810:AAFGObjMdONGYKg_WPc4JyWNiZjijULb68o"

# Initialize the bot with the correct token
bot = telebot.TeleBot(bot_token)

# Dictionary to store user preferences
user_preferences = {}

# Load course data from Excel
def load_course_data():
    try:
        if os.path.exists('course_data.xlsx'):
            df = pd.read_excel('course_data.xlsx')
            logging.info('Course data loaded successfully from Excel')
            return df
        else:
            logging.warning('course_data.xlsx file does not exist')
            return pd.DataFrame()  # Return an empty DataFrame if file does not exist
    except Exception as e:
        logging.error(f'Error loading course data from Excel: {e}')
        return pd.DataFrame()  # Return an empty DataFrame in case of error

# Function to handle /start command
@bot.message_handler(commands=['start'])
def handle_start(message):
    bot.reply_to(message, "Welcome to the NSU Course Notifier Bot! Please use /setpref <course_name> <section> to set your preferred course and section. And then use /check to check the availability")

# Function to handle /setpref command
@bot.message_handler(commands=['setpref'])
def handle_setpref(message):
    if len(message.text.split()) != 3:
        bot.reply_to(message, "Usage: /setpref <course_name> <section>")
        return

    user_id = message.from_user.id
    course_name = message.text.split()[1].upper().strip()  # Normalize input
    section = message.text.split()[2].strip()  # Normalize input
    user_preferences[user_id] = (course_name, section)
    bot.reply_to(message, f"Preference set: {course_name} Section {section}")

# Function to handle /check command
@bot.message_handler(commands=['check'])
def handle_check(message):
    user_id = message.from_user.id
    if user_id not in user_preferences:
        bot.reply_to(message, "You have not set any preferences. Use /setpref to set your preference.")
        return

    course_name, section = user_preferences[user_id]
    course_data = load_course_data()

    # Strip any extra whitespace from course_name and section, and convert course_name to upper case
    course_name = course_name.strip().upper()
    section = section.strip()

    result = course_data[(course_data['Course Name'].str.strip().str.upper() == course_name) & 
                         (course_data['Section'].astype(str).str.strip() == section)]

    if not result.empty:
        seats_available = result.iloc[0]['Seats Available']
        bot.reply_to(message, f"Course: {course_name} Section: {section} Seats Available: {seats_available}")
        # Set up a one-time alert for this course and section if seats are zero
        if int(seats_available) == 0:
            set_seat_alert(user_id, course_name, section)
    else:
        bot.reply_to(message, f"No data found for Course: {course_name} Section: {section}")

# Store alerts for users who need seat notifications
seat_alerts = {}

def set_seat_alert(user_id, course_name, section):
    seat_alerts[user_id] = (course_name, section)
    logging.info(f'Seat alert set for user {user_id} for Course: {course_name} Section: {section}')

# Function to periodically check seat availability
def periodic_check():
    while True:
        try:
            course_data = load_course_data()
            for user_id, (course_name, section) in seat_alerts.items():
                course_name = course_name.strip().upper()  # Ensure consistency
                section = section.strip()
                result = course_data[(course_data['Course Name'].str.strip().str.upper() == course_name) & 
                                     (course_data['Section'].astype(str).str.strip() == section)]
                if not result.empty:
                    seats_available = result.iloc[0]['Seats Available']
                    if int(seats_available) > 0:
                        bot.send_message(user_id, f"Seats available for Course: {course_name} Section: {section} - {seats_available}")
                        del seat_alerts[user_id]  # Remove alert after notifying the user
        except Exception as e:
            logging.error(f'Error in periodic check: {e}')
        time.sleep(600)  # Check every 10 minutes

# Start the periodic check in a separate thread
threading.Thread(target=periodic_check, daemon=True).start()

# Run the bot
bot.polling()


2024-07-03 13:57:07,737 - DEBUG - Starting new HTTPS connection (1): api.telegram.org:443
2024-07-03 13:57:08,521 - INFO - Course data loaded successfully from Excel
2024-07-03 13:57:10,992 - DEBUG - https://api.telegram.org:443 "GET /bot7498318810:AAFGObjMdONGYKg_WPc4JyWNiZjijULb68o/getMe HTTP/11" 200 230
2024-07-03 13:57:10,995 - DEBUG - Starting new HTTPS connection (1): api.telegram.org:443
2024-07-03 13:57:15,473 - DEBUG - https://api.telegram.org:443 "GET /bot7498318810:AAFGObjMdONGYKg_WPc4JyWNiZjijULb68o/getUpdates?offset=1&timeout=20 HTTP/11" 200 1054
2024-07-03 13:57:15,478 - DEBUG - Starting new HTTPS connection (1): api.telegram.org:443
2024-07-03 13:57:15,481 - DEBUG - Starting new HTTPS connection (1): api.telegram.org:443
2024-07-03 13:57:16,566 - DEBUG - https://api.telegram.org:443 "POST /bot7498318810:AAFGObjMdONGYKg_WPc4JyWNiZjijULb68o/sendMessage?chat_id=1878203122&text=Preference+set%3A+CSE445+Section+1&reply_parameters=%7B%22message_id%22%3A+560%7D HTTP/11" 200 602

ConnectTimeout: HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot7498318810:AAFGObjMdONGYKg_WPc4JyWNiZjijULb68o/getUpdates?offset=352215478&timeout=20 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000021F856F6F30>, 'Connection to api.telegram.org timed out. (connect timeout=20)'))